# Discrepency between different randomness for Adapted Optimal Transport Computations

This notebook demonstrates two approaches to computing adapted/nested optimal transport distances between two measures. It includes:
- Running the computations with the same randomness and then with different randomness (by changing the seed).
- Comparing the computed values and timing the computation.


In [1]:
# %% [code]
# Notebook setup: load extensions and import required libraries

%load_ext autoreload
%load_ext line_profiler
%autoreload 2
%matplotlib ipympl

import sys 
from os import path as pt
import numpy as np
import matplotlib.pyplot as plt
import ot
from tqdm import tqdm
import random
import time

# Set the notebook and source paths
import os
notebooks_path = os.path.abspath(os.getcwd()) 
src_path = os.path.abspath(os.path.join(notebooks_path, "../src"))
if src_path not in sys.path:
    sys.path.insert(0, src_path)

# Import custom modules from src
from utils_solver import Lmatrix2paths, adapted_empirical_measure, adapted_wasserstein_squared, quantization, nested, plot_V

from adapted_empirical_measure.AEM_grid import *
from trees.Build_trees_from_paths import *
from trees.TreeAnalysis import *
from awd_trees.Nested_Dist_Algo import *

# Set random seeds for reproducibility
np.random.seed(0)
random.seed(0)
verbose = False

# Create a random seed variable for additional randomness in the notebook
random_seed = np.random.randint(100)
print("Random seed for this run:", random_seed)

Random seed for this run: 44


# Same randomness

In [2]:
# For measure "mu"
print("mu")
L = np.array([[1, 0, 0],
              [2, 4, 0],
              [3, 2, 1]])
n_sample = 1000
normalize = False  # Not used explicitly here
X, A = Lmatrix2paths(L, n_sample, seed=random_seed)

# For measure "nu"
print("nu")
M = np.array([[1, 0, 0],
              [2, 3, 0],
              [3, 1, 2]])
Y, B = Lmatrix2paths(M, n_sample, seed=random_seed)

mu
Cholesky:
[[1 0 0]
 [2 4 0]
 [3 2 1]]
Covariance:
[[ 1  2  3]
 [ 2 20 14]
 [ 3 14 14]]
nu
Cholesky:
[[1 0 0]
 [2 3 0]
 [3 1 2]]
Covariance:
[[ 1  2  3]
 [ 2 13  9]
 [ 3  9 14]]


## Computation with my code

In [3]:
# Compute uniform adapted empirical grid measures with weights
adapted_X, adapted_weights_X = uniform_empirical_grid_measure(X.T, delta_n=0.1, use_weights=True)
adapted_Y, adapted_weights_Y = uniform_empirical_grid_measure(Y.T, delta_n=0.1, use_weights=True)

# Build trees from the adapted paths
adapted_tree_1 = build_tree_from_paths(adapted_X, adapted_weights_X)
adapted_tree_2 = build_tree_from_paths(adapted_Y, adapted_weights_Y)

# Compute the nested (adapted optimal transport) distance and measure execution time
max_depth = get_depth(adapted_tree_1)
start_time = time.time()
distance_pot = compute_nested_distance(
    adapted_tree_1,
    adapted_tree_2,
    max_depth,
    method="solver_lp_pot",
    return_matrix=False,
    lambda_reg=0,
    power=2,
)
elapsed_time_pot = time.time() - start_time

print("Numerical AW_2^2 (Adapted OT):", distance_pot)
print("Elapsed time (Adapted OT): {:.4f} seconds".format(elapsed_time_pot))

Numerical AW_2^2 (Adapted OT): 3.1500599999999985
Elapsed time (Adapted OT): 173.9053 seconds


## With the other code

In [4]:
# Grid projection of k-mean projection
adaptedX = adapted_empirical_measure(X, delta_n = 0.1)
adaptedY = adapted_empirical_measure(Y, delta_n = 0.1)

q2v, v2q, mu_x, nu_y, q2v_x, v2q_x, q2v_y, v2q_y = quantization(adaptedX, adaptedY, markovian=False)

start_time = time.time()
AW_2square, V = nested(mu_x, nu_y, v2q_x, v2q_y, q2v, markovian=False)
elapsed_time_pot = time.time() - start_time

dist_bench = adapted_wasserstein_squared(A, B)
print("Theoretical AW_2^2: ", dist_bench)
print("Numerical AW_2^2: ", AW_2square)
print("Elapsed time (Adapted OT): {:.4f} seconds".format(elapsed_time_pot))

Quantization ......
Number of distinct values in global quantization:  229
Number of condition subpaths of mu_x
Time 0: 1
Time 1: 59
Time 2: 915
Number of condition subpaths of nu_y
Time 0: 1
Time 1: 59
Time 2: 896
Nested backward induction .......


Timestep 0: 100%|██████████| 1/1 [00:00<00:00, 35.48it/s]


Theoretical AW_2^2:  3.0
Numerical AW_2^2:  3.15006
Elapsed time (Adapted OT): 16086.1546 seconds


# Repeat the computations with a different randomness: change seed for nu generation

In [5]:
n_sample = 4000

print("mu")
# For measure mu we keep the same seed as before
L = np.array([[1, 0, 0],
              [2, 4, 0],
              [3, 2, 1]])
X, A = Lmatrix2paths(L, n_sample, seed=random_seed)

print("nu")
# For measure nu, use a different seed by adding 1
M = np.array([[1, 0, 0],
              [2, 3, 0],
              [3, 1, 2]])
Y, B = Lmatrix2paths(M, n_sample, seed=random_seed+1)

mu
Cholesky:
[[1 0 0]
 [2 4 0]
 [3 2 1]]
Covariance:
[[ 1  2  3]
 [ 2 20 14]
 [ 3 14 14]]
nu
Cholesky:
[[1 0 0]
 [2 3 0]
 [3 1 2]]
Covariance:
[[ 1  2  3]
 [ 2 13  9]
 [ 3  9 14]]


In [6]:
adapted_X, adapted_weights_X = uniform_empirical_grid_measure(X.T, delta_n=0.1, use_weights=True)
adapted_Y, adapted_weights_Y = uniform_empirical_grid_measure(Y.T, delta_n=0.1, use_weights=True)

adapted_tree_1 = build_tree_from_paths(adapted_X, adapted_weights_X)
adapted_tree_2 = build_tree_from_paths(adapted_Y, adapted_weights_Y)


max_depth = get_depth(adapted_tree_1)
start_time = time.time()
distance_pot = compute_nested_distance(
    adapted_tree_1,
    adapted_tree_2,
    max_depth,
    method="solver_lp_pot",
    return_matrix=False,
    lambda_reg=0,
    power=2,
)
elapsed_time_pot = time.time() - start_time

print("Numerical AW_2^2 (Adapted OT):", distance_pot)
print("Elapsed time (Adapted OT): {:.4f} seconds".format(elapsed_time_pot))

KeyboardInterrupt: 

In [ ]:
# Grid projection of k-mean projection
adaptedX = adapted_empirical_measure(X, delta_n = 0.1)
adaptedY = adapted_empirical_measure(Y, delta_n = 0.1)

q2v, v2q, mu_x, nu_y, q2v_x, v2q_x, q2v_y, v2q_y = quantization(adaptedX, adaptedY, markovian=False)

start_time = time.time()
AW_2square, V = nested(mu_x, nu_y, v2q_x, v2q_y, q2v, markovian=False)
elapsed_time_pot = time.time() - start_time

dist_bench = adapted_wasserstein_squared(A, B)
print("Theoretical AW_2^2: ", dist_bench)
print("Numerical AW_2^2: ", AW_2square)
print("Elapsed time (Adapted OT): {:.4f} seconds".format(elapsed_time_pot))